In [ ]:
import pandas as pd
import numpy as np
import spacy
import re


In [ ]:
# samo promjenite path ako stavite podatke na drugo mjesto i ne znam zasto, al kad se makne ovaj name argument nece ucitat podatke, pa ako se nekom da otkrit zasto nek proba
file = "friends.train.episode_delim.conll"
unprocessed_data = pd.read_csv(filepath_or_buffer=file, sep=" ", names=["Document ID"])


def load_data_fn(data, file, ) -> pd.DataFrame:
  """
  Rjesava se praznina koje dobijemo kod obicnog pd.read_csv

  PARAMETERS:
    data (pd.Dataframe): pandas dataframe kad ucitas sa pd.read_csv
  """
  with open(file) as d:
    result = []
    for item in d:
      tmp = re.sub("\s", "|", item)
      tmp2 = tmp.split("|")
      lst = []
      for el in tmp2:
        if el != "":
          lst.append(el)
      result.append(lst)
    df = pd.DataFrame(result, columns=["Document ID","Scene ID","Token ID","Word form","Part-of-speech tag","Constituency tag","Lemma","Frameset ID", "Word sense", "Speaker", "Named entity tag", "Entity ID"])
    df.drop(df.tail(1).index,inplace=True)
    df.drop(df.head(1).index,inplace=True)
    df.dropna(inplace=True)
    return df

data = load_data_fn(unprocessed_data, file)


In [ ]:
#data.head(50)

In [ ]:
!pip install nlpaug

     |████████████████████████████████| 399kB 13.0MB/s 


In [ ]:
entities = list()
with open("friends_entity_map.txt") as file:
  for line in file:
    line = line.split("\t", 1)[1][:-1]
    entities.append(line)
print(entities)

['Abby', 'Al', 'Al Kostelic', 'Al Minser', 'Al Pacino', 'Alan', 'Albert Einstein', 'Alex', 'All', 'Amber', 'Amger', 'Andie McDowell', 'Andr', 'Andre', 'Andrea', "Andrea's friend", 'Andrew', 'Angela Delvecchio', 'Annabelle', 'Artelle', 'Ashley', 'Aunt Edna', 'Aunt Iris', 'Aunt Lillian', 'Aunt Phyllis', 'Aunt Syl', 'Aurora', 'Avery', 'Barbara', 'Barry', 'Ben', 'Bernie Spellman', 'Best Man', 'Bethel', 'Betty', 'Big Bully', 'Bill Clinton', "Bill Dreskin's father", 'Billy Dreskin', 'Bing', 'Bishop Tutu', 'Black Bart', 'Bob', 'Bobby Rush', 'Boss Man Bing', 'Brad', 'Braverman', 'Brent Mussberger', 'Brian', 'Brittany', 'Carl', 'Carol Willick', "Carol and Susan's caterer", "Carol's grandmother", 'Caroline', 'Casey', 'Casting Guy', 'Cathy Bates', 'Celia', 'Chandler Bing', "Chandler's date", "Chandler's date's husband", "Chandler's date's husband's secretary", "Chandler's ex-girlfriend", "Chandler's girlfriend", "Chandler's imaginary friend", "Chandler's secretary", 'Charles Bing', 'Chrissy', 'Ch

In [ ]:
import nlpaug
import nlpaug.augmenter.word as naw
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words('english') + entities
print(stops)
aug_synonym = naw.SynonymAug(stopwords = stops)
aug_antonym = naw.AntonymAug(stopwords = stops)
aug_random = naw.RandomWordAug(action = "delete", stopwords = stops)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
data.shape
ran_start = np.random.randint(0,150171)
ran_end = ran_start + 10000

In [ ]:
print(ran_start)

122829


In [ ]:
#kada augmentiramo sa sinonimima onda veci dio dataseta, s antonimima manje
#moraju biti slijedni podatci
num = 5063
augmented_df = data[:num].copy(deep = True)
#augmented_df = data.sample(n=100)
list_aug =  aug_synonym.augment(augmented_df["Lemma"].values.tolist())
augmented_df["Lemma"] = list_aug
#augmented_df["Word form"] = list_aug
#print(augmented_data[:20])
#augmented_df.head(50)

In [ ]:
sp = spacy.load('en_core_web_sm')

In [ ]:
sentence6 = sp(u'compute computer computed computing')
for word in sentence6:
    print(word.text,  word.lemma_)

compute compute
computer computer
computed compute
computing computing


In [9]:

with open("friends.train.episode_delim.conll", "r") as fread, open("file.conll", "a") as fappend:
  cnt = 0
  for line in fread:

    if cnt>=1:
      line2 = line.split()
      #print(line)
      '''
      word = sp(line[3])
      for w in word:
        if w.lemma_ == line[6]:
          print("lema je")
      break
      '''
      if len(line2)<=11:
        fappend.write(line)
        continue
      word_form = sp(augmented_df.iloc[cnt-1]["Word form"])
      lemma = augmented_df.iloc[cnt-1]["Lemma"]
      lemma_list = lemma.split()
      for w in word_form:
        wlemma = w.lemma_
        if lemma != wlemma and lemma.lower() not in stops and re.search('[a-zA-Z]', lemma) and len(lemma_list)==1:
          if wlemma not in "friends" and str(w) not in "friends":
            if str(w)==wlemma:
              line = line.replace(wlemma, lemma, 2)
            else:
              line = line.replace(str(w), lemma, 1)
              line = line.replace(wlemma, lemma, 1)
      fappend.write(line)
    cnt+=1
    if cnt == num:
      break


In [ ]:
frames = [data, augmented_df]

df = pd.concat(frames)

#print(data.shape)
#print(df.shape)

In [ ]:
df['sentence_id'] = -1
cnt = 0
for index, row in df.iterrows():
  if row['Token ID'] == '0':
    cnt += 1
  #print(cnt)
  #print(row['Token ID'])
  df.loc[index, 'sentence_id'] = cnt
df.head()

,Document ID,Scene ID,Token ID,Word form,Part-of-speech tag,Constituency tag,Lemma,Frameset ID,Word sense,Speaker,Named entity tag,Entity ID,sentence_id
1,/friends-s01e01,0,0,There,EX,(TOP(S(NP*),there,-,-,Monica_Geller,*,-,1
2,/friends-s01e01,0,1,'s,VBZ,(VP*,be,-,-,Monica_Geller,*,-,1
3,/friends-s01e01,0,2,nothing,NN,(NP*,nothing,-,-,Monica_Geller,*,-,1
4,/friends-s01e01,0,3,to,TO,(S(VP*,to,-,-,Monica_Geller,*,-,1
5,/friends-s01e01,0,4,tell,VB,(VP*))))),tell,-,-,Monica_Geller,*,-,1


In [ ]:
df.head(50)

,Document ID,Scene ID,Token ID,Word form,Part-of-speech tag,Constituency tag,Lemma,Frameset ID,Word sense,Speaker,Named entity tag,Entity ID,sentence_id
1,/friends-s01e01,0,0,There,EX,(TOP(S(NP*),there,-,-,Monica_Geller,*,-,1
2,/friends-s01e01,0,1,'s,VBZ,(VP*,be,-,-,Monica_Geller,*,-,1
3,/friends-s01e01,0,2,nothing,NN,(NP*,nothing,-,-,Monica_Geller,*,-,1
4,/friends-s01e01,0,3,to,TO,(S(VP*,to,-,-,Monica_Geller,*,-,1
5,/friends-s01e01,0,4,tell,VB,(VP*))))),tell,-,-,Monica_Geller,*,-,1
6,/friends-s01e01,0,5,!,.,*)),!,-,-,Monica_Geller,*,-,1
8,/friends-s01e01,0,0,He,PRP,(TOP(S(NP*),he,-,-,Monica_Geller,*,(284),2
9,/friends-s01e01,0,1,'s,VBZ,(VP*,be,-,-,Monica_Geller,*,-,2
10,/friends-s01e01,0,2,just,RB,(ADVP*),just,-,-,Monica_Geller,*,-,2
11,/friends-s01e01,0,3,some,DT,(NP(NP*,some,-,-,Monica_Geller,*,-,2


In [ ]:
!pip install tabulate

In [ ]:
from tabulate import tabulate

In [ ]:
#df['join'] = df['Document ID'] + '\t' + df['Scene ID'] + '\t' + df['Token ID'] + '\t' + df['Word form'] + '\t' + df['Part-of-speech tag'] + \
            #'\t' + df['Constituency tag'] + '\t' + df['Lemma'] + '\t' + df['Frameset ID'] + '\t' + df['Word sense'] + \
            #'\t' + df['Speaker'] + '\t' + df['Named entity tag'] + '\t' + df['Entity ID']



#alternative
#df['join'] = df['words'].str.cat(df['labels'], sep=' ')
'''
text_file=open("augmented_nan4.txt","w")
for index, g in df.groupby('sentence_id'):
    #out = g.to_dict()
    out = tabulate(g.values.tolist(), stralign="right", showindex=False, tablefmt="plain")
    out = out + "\n"
    #print(out)
    #break
    text_file.write(str(out))
    #out.to_csv('augmented_nan2.txt', index=False, header=None, mode='a')
text_file.close()
'''

'\ntext_file=open("augmented_nan4.txt","w")\nfor index, g in df.groupby(\'sentence_id\'):\n    #out = g.to_dict()\n    out = tabulate(g.values.tolist(), stralign="right", showindex=False, tablefmt="plain")\n    out = out + "\n"\n    #print(out)\n    #break\n    text_file.write(str(out))\n    #out.to_csv(\'augmented_nan2.txt\', index=False, header=None, mode=\'a\')\ntext_file.close()\n'

In [ ]:
g = df.groupby('sentence_id')
def f(gr):
  return gr.append(pd.Series({'new':np.nan}), ignore_index=True)
g.apply(f).reset_index(drop=True)

,Document ID,Scene ID,Token ID,Word form,Part-of-speech tag,Constituency tag,Lemma,Frameset ID,Word sense,Speaker,Named entity tag,Entity ID,sentence_id,new
0,/friends-s01e01,0,0,There,EX,(TOP(S(NP*),there,-,-,Monica_Geller,*,-,1.0,NaN
1,/friends-s01e01,0,1,'s,VBZ,(VP*,be,-,-,Monica_Geller,*,-,1.0,NaN
2,/friends-s01e01,0,2,nothing,NN,(NP*,nothing,-,-,Monica_Geller,*,-,1.0,NaN
3,/friends-s01e01,0,3,to,TO,(S(VP*,to,-,-,Monica_Geller,*,-,1.0,NaN
4,/friends-s01e01,0,4,tell,VB,(VP*))))),tell,-,-,Monica_Geller,*,-,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189928,/friends-s02e17,0,3,be,VBD,(VP*))))),be,-,-,Chandler_Bing,*,-,21000.0,NaN
189929,/friends-s02e17,0,4,",",",",*,",",-,-,Chandler_Bing,*,-,21000.0,NaN
189930,/friends-s02e17,0,5,it,PRP,(NP*),it,-,-,Chandler_Bing,*,-,21000.0,NaN
189931,/friends-s02e17,0,6,be,VBD,(VP*,be,-,-,Chandler_Bing,*,-,21000.0,NaN


In [ ]:
content2=tabulate(g, stralign="right", showindex=False, tablefmt="plain")
text_file=open("output.txt","w")
text_file.write(content2)
text_file.close()

In [ ]:
#pip install transformers

In [ ]:
aug2 = naw.ContextualWordEmbsAug(stopwords=stopwords.words('english'))
augmented_data = aug2.augment(data_list)
print(data_list[:30])
print(augmented_data[:30])

ModuleNotFoundError: ignored